In [ ]:
pip install git+https://github.com/CDCgov/phdi azure-keyvault-secrets

In [ ]:
from pyspark.sql import SparkSession
from phdi.linkage.seed import convert_to_patient_fhir_resources

spark = SparkSession.builder.getOrCreate()

# Set up file client
account_name = "phdidevphi9d194c64"
file_system_name = "source-data"
file_path = "synthetic_patient_mpi_seed_data.parquet"
mpi_incoming_file_path = f"abfss://{file_system_name}@{account_name}.dfs.core.windows.net/{file_path}"

def convert(mpi_incoming_file_path):
    df = spark.read.parquet(mpi_incoming_file_path)
    converted_data = {}
    for row in df.collect():
        iris_id, fhir_bundle = convert_to_patient_fhir_resources(row.asDict())
        converted_data[iris_id] = fhir_bundle
    return converted_data

converted_data = convert(mpi_incoming_file_path)


In [ ]:
from azure.identity import ManagedIdentityCredential
from azure.core.credentials import AccessToken

class spoof_token:
    def get_token(*args, **kwargs):
        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="vault"),
            expires_on=int(time.time())+60*10 # some random time in future... synapse doesn't document how to get the actual time
        )

credential = ManagedIdentityCredential()
credential._credential = spoof_token() # monkey-patch the contents of the private `_credential`

# Set up key vault client
from azure.keyvault.secrets import SecretClient

vault_url = "https://devvault9d194c64.vault.azure.net/"

client = SecretClient(vault_url=vault_url, credential=credential)

# Get client ID and secret for GitHub app registration
client_id = client.get_secret("github-app-client-id").value
client_secret = client.get_secret("github-app-client-secret").value

# Get access token for record linkage container app
import requests

url = "https://login.microsoftonline.com/28cf58df-efe8-4135-b2d1-f697ee74c00c/oauth2/v2.0/token"

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'api://phdi-dev-record-linkage/.default'
}

response = requests.post(url, data=data)

access_token = response.json()['access_token']

# Make request to record linkage container app
url = "https://phdi-dev-record-linkage.victoriousocean-a1ab497b.centralus.azurecontainerapps.io/link-record"

headers = {
    'Authorization': f'Bearer {access_token}'
}

for iris_id, fhir_bundle in converted_data.items():
    data = {
        'bundle': fhir_bundle,
        'external_person_id': iris_id
    }
    requests.post(url, headers=headers, json=data)